In [1]:
import time

import torch

import sys
sys.path.append('../')
from ptsemseg.models import get_model

In [2]:
n_classes = 19
unet_baseline = get_model({'arch': 'unet_baseline'}, n_classes)
unet_alpha0125 = get_model({'arch': 'unet_octconv', 'alpha': 0.125}, n_classes)
unet_alpha0250 = get_model({'arch': 'unet_octconv', 'alpha': 0.250}, n_classes)
unet_alpha0500 = get_model({'arch': 'unet_octconv', 'alpha': 0.500}, n_classes)
unet_alpha0750 = get_model({'arch': 'unet_octconv', 'alpha': 0.750}, n_classes)
unet_alpha0875 = get_model({'arch': 'unet_octconv', 'alpha': 0.875},n_classes)

input_shape = [3, 224, 224]
dummy_input = torch.zeros(input_shape)
dummy_input = dummy_input.unsqueeze(0)

In [3]:
def measure(model, dummy_input, trial_num=50, warmup=5, device='cpu'):
    model.eval()
    model = model.to(device)
    dummy_input = dummy_input.to(device)
    
    for _ in range(warmup):
        with torch.no_grad():
            model(dummy_input)
    
    start = time.time()
    for _ in range(trial_num):
        with torch.no_grad():
            model(dummy_input)
    end = time.time()
    
    average = (end - start) / trial_num
    return average

## CPU

In [4]:
kargs_cpu = {'trial_num': 50, 'warmup': 5, 'device': 'cpu'}

In [5]:
measure(unet_baseline, dummy_input, **kargs_cpu)

/Users/motoki.kimura/work/pytorch-semseg/venv/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/Users/motoki.kimura/work/pytorch-semseg/venv/lib/python3.7/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


0.6572940826416016

In [6]:
measure(unet_alpha0125, dummy_input, **kargs_cpu)

0.882057318687439

In [7]:
measure(unet_alpha0250, dummy_input, **kargs_cpu)

0.808858003616333

In [8]:
measure(unet_alpha0500, dummy_input, **kargs_cpu)

0.5715256261825562

In [9]:
measure(unet_alpha0750, dummy_input, **kargs_cpu)

0.3964305639266968

In [10]:
measure(unet_alpha0875, dummy_input, **kargs_cpu)

0.34745694160461427

## GPU

In [ ]:
kargs_gpu = {'trial_num': 100, 'warmup': 10, 'device': 'cuda:0'}

In [ ]:
measure(unet_baseline, dummy_input, **kargs_gpu)

In [ ]:
measure(unet_alpha0125, dummy_input, **kargs_gpu)

In [ ]:
measure(unet_alpha0250, dummy_input, **kargs_gpu)

In [ ]:
measure(unet_alpha0500, dummy_input, **kargs_gpu)

In [ ]:
measure(unet_alpha0750, dummy_input, **kargs_gpu)

In [ ]:
measure(unet_alpha0875, dummy_input, **kargs_gpu)